## Order of execution of SQL

Let us review the order of execution of SQL. First let us review the order of writing the query.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
val username = System.getProperty("user.name")

username = itv002461


itv002461

In [2]:
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Windowing Functions").
    master("yarn").
    getOrCreate

username = itv002461
spark = org.apache.spark.sql.SparkSession@42cecd7e


org.apache.spark.sql.SparkSession@42cecd7e

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

1. **SELECT**
2. **FROM**
3. **JOIN** or **OUTER JOIN** with **ON**
4. **WHERE**
5. **GROUP BY** and optionally **HAVING**
6. **ORDER BY**

Let us come up with a query which will compute daily revenue using COMPLETE or CLOSED orders and also sorted by order_date.

In [3]:
%%sql

USE itv002461_retail

Waiting for a Spark session to start...

++
||
++
++



In [4]:
%%sql

SELECT o.order_date,
  round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date
ORDER BY o.order_date
LIMIT 10

+--------------------+--------+
|          order_date| revenue|
+--------------------+--------+
|2013-07-25 00:00:...|31547.23|
|2013-07-26 00:00:...|54713.23|
|2013-07-27 00:00:...|48411.48|
|2013-07-28 00:00:...|35672.03|
|2013-07-29 00:00:...| 54579.7|
|2013-07-30 00:00:...|49329.29|
|2013-07-31 00:00:...|59212.49|
|2013-08-01 00:00:...|49160.08|
|2013-08-02 00:00:...|50688.58|
|2013-08-03 00:00:...|43416.74|
+--------------------+--------+



In [7]:
%%sql

SELECT o.order_date,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date
    HAVING round(sum(oi.order_item_subtotal), 2) >= 50000
ORDER BY order_date
LIMIT 10

+--------------------+--------+
|          order_date| revenue|
+--------------------+--------+
|2013-07-26 00:00:...|54713.23|
|2013-07-29 00:00:...| 54579.7|
|2013-07-31 00:00:...|59212.49|
|2013-08-02 00:00:...|50688.58|
|2013-08-06 00:00:...|57843.89|
|2013-08-12 00:00:...|59014.74|
|2013-08-17 00:00:...|63226.83|
|2013-08-24 00:00:...|52650.15|
|2013-09-05 00:00:...|59942.43|
|2013-09-06 00:00:...| 61976.1|
+--------------------+--------+



However order of execution is typically as follows.

1. **FROM**
2. **JOIN** or **OUTER JOIN** with **ON**
3. **WHERE**
4. **GROUP BY** and optionally **HAVING**
5. **SELECT**
6. **ORDER BY**

As **SELECT** is executed before **ORDER BY** clause, we will not be able to refer the aliases defined in **SELECT** caluse in other clauses except for **ORDER BY** in most of the traditional databases. However, in Spark we can specify the aliases defined in **SELECT** in **HAVING** as well as **ORDER BY**.

```{error}
This will fail as revenue which is an alias defined in **SELECT** cannot be used in **WHERE**.
```

In [8]:
%%sql

SELECT o.order_date,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
    AND revenue >= 50000
GROUP BY o.order_date
ORDER BY order_date
LIMIT 10

               :  +- Su...


Magic sql failed to execute with error: 
cannot resolve '`revenue`' given input columns: [oi.order_item_quantity, o.order_customer_id, oi.order_item_id, o.order_date, oi.order_item_order_id, oi.order_item_product_price, oi.order_item_subtotal, oi.order_item_product_id, o.order_id, o.order_status]; line 1 pos 202;
'GlobalLimit 10
+- 'LocalLimit 10
   +- 'Sort ['order_date ASC NULLS FIRST], true
      +- 'Aggregate ['o.order_date], ['o.order_date, 'round('sum('oi.order_item_subtotal), 2) AS revenue#114]
         +- 'Filter (order_status#118 IN (COMPLETE,CLOSED) && ('revenue >= 50000))
            +- Join Inner, (order_id#115 = order_item_order_id#120)
               :- SubqueryAlias `o`
               :  +- SubqueryAlias `itv002461_retail`.`orders`
               :     +- HiveTableRelation `itv002461_retail`.`orders`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, [order_id#115, order_date#116, order_customer_id#117, order_status#118]
               +- SubqueryAlias `oi`
           

```{note}
This will work as revenue which is an alias defined in **SELECT** can be used in **HAVING** as well as **ORDER BY**.
```

In [9]:
%%sql

SELECT o.order_date,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date
    HAVING revenue >= 50000
ORDER BY order_date,
    revenue DESC
LIMIT 10

+--------------------+--------+
|          order_date| revenue|
+--------------------+--------+
|2013-07-26 00:00:...|54713.23|
|2013-07-29 00:00:...| 54579.7|
|2013-07-31 00:00:...|59212.49|
|2013-08-02 00:00:...|50688.58|
|2013-08-06 00:00:...|57843.89|
|2013-08-12 00:00:...|59014.74|
|2013-08-17 00:00:...|63226.83|
|2013-08-24 00:00:...|52650.15|
|2013-09-05 00:00:...|59942.43|
|2013-09-06 00:00:...| 61976.1|
+--------------------+--------+

